# Ron Day 2 Homework - Group Coding Challenge
Exploring the LJ Potential and Cutoffs

In [ ]:
import math

In [ ]:
def calculate_LJ(r_ij): 
    """
    The LJ interaction energy between two particles.
    
    Computes the pairwise Lennard jones interaction energy based on the separation distance in reduced units.
    
    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
        
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.
    
    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
fig = plt.figure ()
ax = fig.add_subplot(111)
plt.ylim(-2,50)
fig.show()

In [ ]:
for i in range(1, 51):
    r = i * 0.1
    LJ_energy = calculate_LJ(r)
    ax.plot(r,LJ_energy, 'ob')
    #print(f'{r} \t  {LJ_energy}')

## Is 3sigma a good cutoff value or not?
If we plug a distance of 3sigma into the Lennard Jones equation, the terms reduce to 4E[(1/3)^12 - (1/3)^6] = -0.0055* Epsilon. Since Epsilon in SI units is typically a tiny smaller decimal number, that product works out to be a miniscule value, on an order of magnitude essentially equivalent to 0. Therefore, I agree that a cutoff of 3sigma should be satisfactory.


In [ ]:
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates.
    
    Parameters
    ----------
    coord1, coord2 : list
        The atomic coordinates [x, y, z]
        
    box_length : float, optional
        The box length. This function assumes box is a cube.
        
    Returns
    -------
    distance: float
        The distance between the two atoms
    
    """
    #Do periodic boundary corrections if given a box_length
    if box_length is not None:
        #Distance = sqrt(sum of square differences of each dimension)
        #initialize the sum of square differences to 0
        sum_square_diff = 0
        
        #Iterate through dimensions
        for i in range(len(coord1)):
            
            #Find the raw distance between the two coordinates in this dimension
            dim_dist_uncorrected = math.fabs(coord1[i] - coord2[i])
            
            #Periodic boundary corrections
            #If raw distance is less than half the box_length, no corrections are needed
            if dim_dist_uncorrected <= box_length / 2:
                dim_dist_corrected = dim_dist_uncorrected
            
            #If raw distance is greater than half the box length and less than one whole box length, correct accordingly
            elif (dim_dist_uncorrected > box_length / 2 and dim_dist_uncorrected <= box_length):
                dim_dist_corrected = box_length - dim_dist_uncorrected
            
            #If raw distance is greater than one whole box length, correct accordingly
            else:
                dim_dist_corrected = dim_dist_uncorrected - box_length * round(dim_dist_uncorrected / box_length)
            
            #Add the square difference to the total sum
            sum_square_diff += (dim_dist_corrected)**2
        
        #Calculate distance after finding the sum of square differences
        distance = math.sqrt(sum_square_diff)
        
    #Otherwise assume no periodic boundaries    
    else:
        sum_square_diff = 0
    
        for i in range(len(coord1)):
            sum_square_diff += (coord1[i] - coord2[i])**2

        distance = math.sqrt(sum_square_diff)

    return distance

In [ ]:
#Defining a function for the tail correction

def calculate_tail_correction(particles, box, r_cutoff):
    """
    This function calculates the tail correction
    
    Parameters
    ----------
    particles : int
        The number of particles in the sample
        
    volume : float
        The volume of the sample
        
    r_cutoff : float
        The cutoff radius
    
    Returns
    -------
    tail_correction_factor: float
            The tail correction factor in reduced units
    
    """
    r3_term = math.pow(1/r_cutoff, 3)
    r9_term = math.pow(r3_term, 3)
    
    tail_correction_factor = ((8* math.pi * (particles ** 2)) / (3 * volume) ) * (((1/3) * (r9_term)) - r3_term)
    
    return tail_correction_factor

In [ ]:
def calculate_total_pair_energy(coordinates, box_length, cutoff):
    """
    Calculate the total Lennard Jonest energy of a system of particles.
    
    Parameters
    ----------
    coordinates : list
        Nested list containing particle coordinates.
        
    Returns
    -------
    total_energy : float
        The total pairwise Lennard jonest energy of the system of particles.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

In [ ]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length

In [ ]:
import os

In [ ]:
%cd ../..

In [ ]:
file_path = os.path.join('lj_sample_configurations', 'lj_sample_config_periodic1.txt')

coordinates, box_length = read_xyz(file_path)

In [ ]:
%time calculate_total_pair_energy(coordinates, 1)

In [ ]:
%time calculate_total_pair_energy(coordinates, 2)

In [ ]:
calculate_total_pair_energy(coordinates, 10, 3)

## Flow of Calculations
1. generate initial system state m
1. choose an atom with uniform probability from stat m
1. propose a new state n by translating the particle with a uniform random displacement in each direction
1. calculate the energy change for the particle
1. accept the reject new state


In [ ]:
def accept_or_reject(delta_e, beta):
    """
    Accept or reject based on change in energy and temperature.
    """
    
    if delta_e <= 0:
        accept = True
    else:
        random_number = random.random()
        p_acc = math.exp(-beta*delta_e)
        
        if random_number < p_acc:
            accept = True
        else:
            accept = False
    
    return accept

In [ ]:
delta_energy = -1
beta = 1
assert accept_or_reject(delta_energy, beta) is True

In [ ]:
delta_energy = 0
beta = 1
assert accept_or_reject(delta_energy, beta) is True

In [ ]:
import random

random.seed(0)
random.random()

In [ ]:
random.seed(0)
delta_energy = 1
beta = 1
assert accept_or_reject(delta_energy, beta) is False

In [ ]:
random.seed(1)
random.random()


In [ ]:
random.seed(1)
delta_energy = 1
beta = 1
assert accept_or_reject(delta_energy, beta) is True

In [ ]:
# Unset random seed
random.seed()

In [ ]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system).
    
    Parameters
    ----------
    coordinates : list
        the coordinates for all the particles in the system
        
    i_particle : integer
        The particle index for which to calculate the energy.
        
    box_length : float
        THe length of the simulation box.
    
    cutoff : float
        The cutoff distance for our simulation. Beyond this distance, interactions are not calculated.
        
    Returns
    -------
    e_total : float
        The pairwise interaction energy of the i-th particle with all other particles in the system.
    """
    
    e_total = 0
    
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
        if i_particle != j_particle:
            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position)

            if rij < cutoff:
                #print(rij, i_particle, j_particle, j_position)
                e_pair = calculate_LJ(rij)
                e_total += e_pair

    
    return e_total

In [ ]:
coordinates = [[0,0,0], [0,0,2**(1/6)], [0,0,2*(2**(1/6))]]
#calculate_pair_energy(coordinates, 1, 10, 3)
assert calculate_pair_energy(coordinates, 1, 10, 3) == -2

assert c

In [ ]:
cd msse-bootcamp/

## Day 3 Class Work - Simulation Loop

In [ ]:
calculate_tail_correction(800, 10, 3)

In [ ]:
import os

# Set simulation arameters
reduced_temperature = 1.5
num_steps = 10000
max_displacement = 0.1
cutoff = 3

# Reporting information
freq = 1000
steps = []
energies = []
all_coordinates = []

# Calculated quantities
beta = 1/reduced_temperature

# Read initial coordinates
file_path = os.path.join('lj_sample_configurations', 'lj_sample_config_periodic1.txt')
coordinates, box_length = read_xyz(file_path)
num_particles = len(coordinates)
print(num_particles)
# Calculated based on simulation inputs
total_energy = calculate_total_pair_energy(coordinates, box_length, cutoff)
print(total_energy)
total_energy += calculate_tail_correction(num_particles, box_length**3, cutoff)
print(total_energy)
for step in range(num_steps):
    
    # 1. Randomly pick 1 of num_particles particles.
    random_particle = random.randrange(num_particles)
    
    # 2. Calculate the interaction energy of the selected particle with the system. Store this value.
    current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    
    # 3. Generate a random x, y, z displacement range (-max_displacement, max_displacement) - uniform distribution
    x_rand = random.uniform(-max_displacement, max_displacement)
    y_rand = random.uniform(-max_displacement, max_displacement)
    z_rand = random.uniform(-max_displacement, max_displacement)
    
    # 4.Modify the coordinate of selected particle by displacements.
    coordinates[random_particle][0] += x_rand
    coordinates[random_particle][1] += y_rand
    coordinates[random_particle][2] += z_rand
    # 5. Calclulate the new interaction energy of the new particle, store this value.
    proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
    
    # 6. Calculate energy change and decide if we accept the move.
    delta_energy = proposed_energy - current_energy
    
    accept = accept_or_reject(delta_energy, beta)
    
    # 7. If we accept, keep movement. If not, revert to old position.
    if accept:
        total_energy += delta_energy
    else:
        # Move is not accepted, roll back coordinates
        coordinates[random_particle][0] -= x_rand
        coordinates[random_particle][1] -= y_rand
        coordinates[random_particle][2] -= z_rand
    
    # 8. Print the energy and store the coordinates at certain intervals.
    if step % freq == 0:
        print(step, total_energy/num_particles)
        steps.append(step)
        energies.append(total_energy/num_particles)
        all_coordinates.append(coordinates)
    


In [ ]:
energies

In [ ]:
import matplotlib.pyplot as plt

%matplotlib notebook

fig  = plt.figure()
ax = fig.add_subplot(111)
ax = (delta_e, energies)